# nbdev

## TO DO IN THIS NOTEBOOK
- [ ] Restructure as a much tighter review of nbdev and how it can help in the development of a package to be used for our dashboard project
- [x] Be sure to mention the challenges of using nbdev, notably the need to be careful about what you export to packages (do we keep discussion of literate programming in here?)
- [ ] Get to the point where a first version of the dashboard package is created, but note the complexity in the handling of all the settings since we require observer functions to handle all the individual settings (since they are separate widgets).  Do we mention the need to code up validation of the settings (e.g. - We need to ensure the degree of the fit was reasonable give the number of data points, etc.)?
- [x] Add hyperlinks to the nbdev documentation and other resources

## What is nbdev?

- [nbdev](https://nbdev.fast.ai) - a notebook-driven development platform. Simply write notebooks with lightweight markup and get high-quality documentation, tests, continuous integration, and packaging. 

- ["modular" nbdev](https://nbdev.fast.ai/tutorials/modular_nbdev.html) - makes use of only the most basic features of nbdev. It will allow us take any standalone notebook and turn it into a Python script or module. Modular nbdev will allow us to bridge the gap between messy, exploratory tutorial notebooks and clean, well-organized Python modules that we can import from. 

The following introduction to "modular nbdev" is a much simpler than the [nbdev tutorial](https://nbdev.fast.ai/tutorials/tutorial.html), because we won't create a continuous integration Python package that can be uploaded to PyPi or Anaconda. We will simply use notebooks to create clean python modules that we can use to build our dashboards and web apps. From here on out, when we mention nbdev, we are really referring to modular nbdev. 

## What will we learn in this notebook?

### Export code to modules with nb_export
- `#| default_exp` directive to signal which module to export to
- `#| export` directive to signal that a cell is to be included in the export
- `nbdev.export.nb_export` with the name of the notebook and export directory
- benefits and challenges of nbdev

### End Goal

The end goal of this tutorial notebook is to understand how we can use nbdev to create a `dashboard` python module that contains all the files we need for our dashboard.  We can use some of the cells in this notebook to create a python module that we can import to other notebooks. 

## nbdevExports

### Benefits

The real benefit of being able to exclude code cells lies in the ability to tell the story of how your code came into it's final form, whether for a tutorial or just because you value documenting code design and decision making. Instead of cleaning up our notebooks when we have achieved your coding goal, we can leave in all the valuable information we left behind along the way. This is helpful for another developer or our future-selves who may be trying to read our code. We don't have to hold as many variables in their head, because we can see the steps broken down and read the intermediate values without having to use a debugger.

- show intermediate steps for clarity and testing
- document failed attempts and design decisions
- develop iteratively without having to "clean" code

### #| default_exp

The `default_exp` directive is required of all notebooks used by nbdev. Notice that nbdev directives all start with "#|". 
This requires that we add a default export directive to our notebook. It tells nbdev what module to export to. You can also give it a “dotted module names” as is done with python packages. For example, a default export to `some.module` will create a file called `some/module.py`.

### #| export

This directive tells nbdev that we want to export this code to the module specified by `default_exp`. Notice that we need to export everything that we want to end up in the python file, so we can't forget to export our imports.

**NOTE:** When performing an export, the cell can only contain imports OR some other Python commands, but not both.  This is due to the way `nbdev` parses the cells to analyze for documentation.

### Challenges

The great benefit of using nbdev for development can also be the most challening part of using nbdev - making sure we have not forgotten to export all the right cells to our resulting Python file. If you ever get a confusing or unexpected error on importing a module created with nbdev, it's very likely you forgot to export one of your code cells. 

## Create `dashboard` module

How we will use nbdev to create a python module - `dashboard` - that we will use to create modules that contain files we will use improve our development experience.

The first thing we need to do is create a default export statement.

In [ ]:
# TODO: Add default export directive for the dashboard module

### %answer line magic

Below is the code that defines the cell magic function called `exception`. Often in tutorials, we want to demonstrate an intentional error. The default behavior of a Jupyter Notebook is to print the full traceback, which can be quite lengthy and distracting. In addition, if the participant wants to "Restart and Run All Cells" due to some problem they encountered, the execution will get hung up where the error is. In our case, when running a cell with an error for demonstrative purposes, we want to display the error, but we don't want to display the traceback or prevent the execution of the next cell. This cell magic achieves this desired behavior.

In [ ]:
# TODO: add the missing export directive
from IPython.core.magic import register_line_magic, register_cell_magic, needs_local_scope
from IPython import get_ipython
import re, sys, os

In [ ]:
#| export
@register_cell_magic('exception')
def exception(line, cell):
    ip = get_ipython()
    try:
        exec(cell, None, ip.user_ns)
    except Exception as e:
        etype, value, tb = sys.exc_info()
        value.__cause__ = None  # suppress chained exceptions
        ip._showtraceback(etype, value, ip.InteractiveTB.get_exception_only(etype, value))

As I said, the benefit of literate programming is that we can test out this function in-line. We don't have to delete the exploratory or explanitory code. The next line shows what happens when we use the cell magic. Notice that there isn't an export directive at the top, so this breaking error isn't exported to our python module.

In [ ]:
%%exception

5 / 0

You can compare that output to the output below, which includes a traceback by default because the cell magic wasn't used.

In [ ]:
5 / 0

Isn't nbdev great! Instead of playing with toy examples, we can build a sophisticated, clean Python application from a series of tutorial-style notebooks. All the python modules and packages in this tutorial were produced from the tutorial notebooks.

### %answer line magic

The code below allows us to import code from the nbdev-generated python files like the one we loaded above. You can think of this function working like the `%load` line magic, which pulls in code from an external file. But instead of loading the entire file, this function loads only the python code that came from an exported notebook cell. 

In [ ]:
#| export
@register_line_magic('answer')
def answer(inputs):
    '''
    This is a cell magic called answer that allows tutorial goers to import the correct answer from the key. 
    '''
    words = []
    for word in inputs.split(' '):
        if not word.startswith('#') and len(word) != 0:
            words.append(word)
        else:
            break
    
    flag = False
    if len(words) == 2:
        if words[1] == '-e':
            flag = True
        else:
            filepath = words[0]
            cell_number = int(words[1])

            with open(filepath, 'r') as file:
                lines = file.readlines()

            pattern = r'# %%\s+(.+)\s+(\d+)'
            start_line = None
            end_line = None

            for i, line in enumerate(lines):
                if re.match(pattern, line):
                    match = re.search(pattern, line)
                    if match and int(match.group(2)) == cell_number:
                        start_line = i + 1
                        break
            if start_line is not None:
                for i in range(start_line, len(lines)):
                    if re.match(pattern, lines[i]):
                        end_line = i
                        break
                else:
                    end_line = len(lines)

            if start_line is not None and end_line is not None:
                code_chunk = f"#| export\n# %answer {inputs}\n\n" + ''.join(lines[start_line:end_line])
                code_chunk = code_chunk.rstrip("\n")
                get_ipython().set_next_input(code_chunk, replace=True)
            else:
                raise Exception(f"Cell number {cell_number} not found in the Python file.")
        
    if len(words) == 1 or words[1] == '-e':
        filepath = words[0]
        with open(filepath, 'r') as file:
            lines = file.readlines()
        code_chunk = ''.join(lines[:])
        if flag:
            code_chunk = f"# %%export {filepath}\n\n" + code_chunk
        else: 
            code_chunk = f"# %answer {filepath}\n\n" + code_chunk
        get_ipython().set_next_input(code_chunk, replace=True)
    
    with open(filepath, 'r') as file:
        lines = file.readlines()

If at any point in this tutorial you have trouble coming up with an answer on your own, you can always import it from the answer key.

### Pull from exported module

This will be the case when we are pulling in an answer to a cell that starts with `#| export`. The path to the answer key files is the same as the default, but is preceeded by the `key` directory. For example, the next line will load the code from the 6th cell of this notebook, which is the import statements. 

In [ ]:
# %answer key/dashboard/__init__.py 6

### Pull from file

Sometimes we will want to check an answer from a cell that isn't a module exported by nbdev. In this case, we will provide the file path only.

**In both cases, these paths will be provided for you in comments.**

## nb_export

Next we can use the nb_export function to compile the exported cells into a python file. The second parameter tells nbdev which package to export the module to. Let's go ahead and export this notebook to the `dashboard` package. 

> **IMPORTANT**: do not forget to save before exporting the notebook, or your most recent changes will not make it into the py file. It is also a good habit to "Restart Kernel and Clear All Outputs" and then save your notebook at the same time as you export. That way, when you get around to committing your code, your git diff isn't filled up with a bunch of serial numbers and other unintelligible things.

In [ ]:
from nbdev.export import nb_export

nb_export('03_nbdev.ipynb', 'dashboard')

Usually, we will put `nb_export` at the end of our file for convenience, but we will continue on from here.

Another way to verify that this worked is to import the function we just wrote to the python file. Can you guess what that import statement would look like?

In [ ]:
# %answer 03/01.py

# import ...

So for most modules, the import statement would look like `dashboard.__init__` but the `__init__` file is special, and will be imported any time we make an import from `dashboard` package.

Great job! Now we know the basic idea of what nbdev does. Go ahead and use the `%load` line magic inspect the exported file below.

In [ ]:
# %answer key/03/02.py

# %load pkg/module.py

There are a few thing to notice here. One is that we get a warning about this file being autogenerated. That is, if we make changes to `dashboard/__init__.py`, those changes will be overwritten the next time we run `nb_export`.

The second thing to notice is that there is a handy comment to tell us where the code came from. `# %% ../_nbdev.ipynb 9` tells us that the code came from the 9th code cell. We will use this information to write our next magic function.

### The End

We will not go over the code below. Proceed to the next notebook.
___

In [ ]:
#| export
@register_cell_magic('export')
def export(line, cell=None):
    line_args = line.split()
    export_filepath = None

    if len(line_args):
        export_filepath = line_args[0]
        directory = os.path.dirname(export_filepath)
        os.makedirs(directory, exist_ok=True)
        with open(export_filepath, 'w') as file:
            file.write(cell)
        print(f"exported to {export_filepath}")
            
    processed_lines = []
    for line in cell.split('\n'):
        comment_match = re.search(r'#', line)
        if comment_match:
            line = line[comment_match.start():]
            processed_lines.append(line)

    processed_cell = '\n'.join(processed_lines)
    if len(line_args):
        processed_cell = '# %answer ' + line_args[0] + '\n\n' + processed_cell
    get_ipython().set_next_input(processed_cell, replace=True)